# OpenAI Functions

These output parsers use OpenAI function calling to structure its outputs. This means they are only usable with models that support function calling. There are a few different variants:

- JsonOutputFunctionsParser: Returns the arguments of the function call as JSON
- PydanticOutputFunctionsParser: Returns the arguments of the function call as a Pydantic Model
- JsonKeyOutputFunctionsParser: Returns the value of specific key in the function call as JSON
- PydanticAttrOutputFunctionsParser: Returns the value of specific key in the function call as a Pydantic Model


In [38]:
from langchain_community.utils.openai_functions import (
    convert_pydantic_to_openai_function,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

In [5]:
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


openai_functions = [convert_pydantic_to_openai_function(Joke)]

In [6]:
model = ChatOpenAI(temperature=0)

In [11]:
prompt = ChatPromptTemplate.from_messages(
    [("system", "You are helpful assistant"), ("user", "{input}")]
)

## JsonOutputFunctionsParser

In [13]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [14]:
parser = JsonOutputFunctionsParser()

In [15]:
chain = prompt | model.bind(functions=openai_functions) | parser

In [16]:
chain.invoke({"input": "tell me a joke"})

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}

In [17]:
for s in chain.stream({"input": "tell me a joke"}):
    print(s)

{}
{'setup': ''}
{'setup': 'Why'}
{'setup': 'Why don'}
{'setup': "Why don't"}
{'setup': "Why don't scientists"}
{'setup': "Why don't scientists trust"}
{'setup': "Why don't scientists trust atoms"}
{'setup': "Why don't scientists trust atoms?", 'punchline': ''}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything!'}


## JsonKeyOutputFunctionsParser

This merely extracts a single key from the returned response. This is useful for when you want to return a list of things.

In [21]:
from typing import List

from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [31]:
class Jokes(BaseModel):
    """Jokes to tell user."""

    joke: List[Joke]
    funniness_level: int

In [32]:
parser = JsonKeyOutputFunctionsParser(key_name="joke")

In [33]:
openai_functions = [convert_pydantic_to_openai_function(Jokes)]
chain = prompt | model.bind(functions=openai_functions) | parser

In [34]:
chain.invoke({"input": "tell me two jokes"})

[{'setup': "Why don't scientists trust atoms?",
  'punchline': 'Because they make up everything!'},
 {'setup': 'Why did the scarecrow win an award?',
  'punchline': 'Because he was outstanding in his field!'}]

In [35]:
for s in chain.stream({"input": "tell me two jokes"}):
    print(s)

[]
[{}]
[{'setup': ''}]
[{'setup': 'Why'}]
[{'setup': 'Why don'}]
[{'setup': "Why don't"}]
[{'setup': "Why don't scientists"}]
[{'setup': "Why don't scientists trust"}]
[{'setup': "Why don't scientists trust atoms"}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': ''}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything!'}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything!'}, {}]
[{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything!'}, {'

## PydanticOutputFunctionsParser

This builds on top of `JsonOutputFunctionsParser` but passes the results to a Pydantic Model. This allows for further validation should you choose.

In [41]:
from langchain.output_parsers.openai_functions import PydanticOutputFunctionsParser

In [42]:
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


parser = PydanticOutputFunctionsParser(pydantic_schema=Joke)

In [44]:
openai_functions = [convert_pydantic_to_openai_function(Joke)]
chain = prompt | model.bind(functions=openai_functions) | parser

In [45]:
chain.invoke({"input": "tell me a joke"})

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')